In [213]:
import datetime
import operator
import os
import sys
import time
import pyspark
from operator import add
from math import radians, cos, sin, asin, sqrt
start = time.time()

In [214]:
def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findN(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if any(map(lambda x: x.contains(p), zones.geometry[idx])):
            return zones['neighborhood'][idx]
    return -1

In [215]:
def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones('neighborhoods.geojson')
    
    for line in parts[1:]:
        fields = line.strip('').split(',')
        if fields ==['']: continue
        if all((fields[5],fields[6],fields[9],fields[10])):
            for i in [5,6,9,10]:
                fields[i] = fields[i].strip('').split("\"")[1]
            pickup_location  = geom.Point(proj(float(fields[6]), float(fields[5])))
            dropoff_location = geom.Point(proj(float(fields[10]), float(fields[9])))
            pickup_zone = findN(pickup_location, index, zones)
            dropoff_zone = findN(dropoff_location, index, zones)

            if pickup_zone>=0 and dropoff_zone>=0:
                yield ((dropoff_zone, pickup_zone), 1)

In [216]:
trips = sc.textFile('/home/satya/BDM_dataset/Citibike/Citibike.csv')
#output = trips.mapPartitions(mapToZone)
output = mapToZone(trips.take(100))
output1 = sc.parallelize(output)
print (time.time()-start)/60.0

[((u'Stuyvesant Town', u'Midtown'), 1), ((u'Chelsea', u'Chelsea'), 1), ((u'East Village', u'East Village'), 1), ((u'Upper West Side', u'Lower East Side'), 1), ((u'Lower East Side', u'Greenwich Village'), 1), ((u'Midtown', u'Stuyvesant Town'), 1), ((u'East Village', u'East Village'), 1), ((u'Financial District', u'Financial District'), 1), ((u'Greenwich Village', u'East Village'), 1), ((u"Hell's Kitchen", u'Theater District'), 1), ((u'Financial District', u'Tribeca'), 1), ((u'Lower East Side', u'Nolita'), 1), ((u'Theater District', u"Hell's Kitchen"), 1), ((u'Stuyvesant Town', u'Stuyvesant Town'), 1), ((u'Theater District', u"Hell's Kitchen"), 1), ((u'SoHo', u'Midtown'), 1), ((u'Midtown', u'Theater District'), 1), ((u"Hell's Kitchen", u'Chelsea'), 1), ((u'Chelsea', u'Chelsea'), 1), ((u'Midtown', u'Greenwich Village'), 1), ((u'East Village', u'Brooklyn Heights'), 1), ((u'Brooklyn Heights', u'Chelsea'), 1), ((u'Midtown', u'Upper West Side'), 1), ((u'Chelsea', u'Theater District'), 1), ((u